In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('Train.csv')
df.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,1,D,Flight,4,2,177,3,low,F,44,1233,1
1,2,F,Flight,4,5,216,2,low,M,59,3088,1
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1


In [ ]:
df.shape

(10999, 12)

In [ ]:
df.columns

Index(['ID', 'Warehouse_block', 'Mode_of_Shipment', 'Customer_care_calls',
       'Customer_rating', 'Cost_of_the_Product', 'Prior_purchases',
       'Product_importance', 'Gender', 'Discount_offered', 'Weight_in_gms',
       'Reached.on.Time_Y.N'],
      dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10999 entries, 0 to 10998
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   10999 non-null  int64 
 1   Warehouse_block      10999 non-null  object
 2   Mode_of_Shipment     10999 non-null  object
 3   Customer_care_calls  10999 non-null  int64 
 4   Customer_rating      10999 non-null  int64 
 5   Cost_of_the_Product  10999 non-null  int64 
 6   Prior_purchases      10999 non-null  int64 
 7   Product_importance   10999 non-null  object
 8   Gender               10999 non-null  object
 9   Discount_offered     10999 non-null  int64 
 10  Weight_in_gms        10999 non-null  int64 
 11  Reached.on.Time_Y.N  10999 non-null  int64 
dtypes: int64(8), object(4)
memory usage: 1.0+ MB


In [ ]:
df = df.rename(columns={'Reached.on.Time_Y.N':'Reached_on_Time_Y_N', 'Weight_in_gms':'Weight_in_grams'})
df.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_grams,Reached_on_Time_Y_N
0,1,D,Flight,4,2,177,3,low,F,44,1233,1
1,2,F,Flight,4,5,216,2,low,M,59,3088,1
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1


In [ ]:
df.describe()

,ID,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_grams,Reached_on_Time_Y_N
count,10999.00000,10999.000000,10999.000000,10999.000000,10999.000000,10999.000000,10999.000000,10999.000000
mean,5500.00000,4.054459,2.990545,210.196836,3.567597,13.373216,3634.016729,0.596691
std,3175.28214,1.141490,1.413603,48.063272,1.522860,16.205527,1635.377251,0.490584
min,1.00000,2.000000,1.000000,96.000000,2.000000,1.000000,1001.000000,0.000000
25%,2750.50000,3.000000,2.000000,169.000000,3.000000,4.000000,1839.500000,0.000000
50%,5500.00000,4.000000,3.000000,214.000000,3.000000,7.000000,4149.000000,1.000000
75%,8249.50000,5.000000,4.000000,251.000000,4.000000,10.000000,5050.000000,1.000000
max,10999.00000,7.000000,5.000000,310.000000,10.000000,65.000000,7846.000000,1.000000


#### 이상치 제거

In [ ]:
Q1 = df['Discount_offered'].quantile(0.25)
Q3 = df['Discount_offered'].quantile(0.75)

IQR = Q3 - Q1
high_limit = Q3 + IQR * 1.5
low_limit = Q1 - IQR * 1.5 # -5.0

df = df[df['Discount_offered'] < high_limit]
df.shape

(8737, 12)

In [ ]:
train = df[['Warehouse_block', 'Mode_of_Shipment', 'Customer_care_calls','Customer_rating', 'Cost_of_the_Product', 'Prior_purchases', 'Product_importance', 'Gender', 'Discount_offered', 'Weight_in_grams']].copy()
target = df[['Reached_on_Time_Y_N']].copy()

#### 수치형 데이터 스케일링

In [ ]:
numeric_col = train.select_dtypes(include=['int']).columns
numeric_col

Index(['Customer_care_calls', 'Customer_rating', 'Cost_of_the_Product',
       'Prior_purchases', 'Discount_offered', 'Weight_in_grams'],
      dtype='object')

In [ ]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
train[numeric_col] = ss.fit_transform(train[numeric_col])
train[numeric_col]

,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_grams
3,-0.972147,0.005764,-0.786512,0.239733,1.212500,-1.718296
5,-0.972147,-1.412904,-1.077646,-0.420175,1.803398,-1.570146
6,-0.972147,0.715099,0.752342,-0.420175,-0.855642,-0.981252
8,-0.972147,0.715099,-1.327190,-0.420175,1.507949,-1.296070
10,-0.972147,0.715099,-0.516173,-1.080084,1.803398,-0.662113
...,...,...,...,...,...,...
10994,-0.113596,-1.412904,0.793933,0.899642,-1.446539,-1.495454
10995,-0.113596,-1.412904,0.378027,0.899642,0.030705,-1.675086
10996,0.744955,0.715099,0.585980,0.899642,-0.560193,-1.731876
10997,0.744955,-0.703570,0.190869,1.559551,-1.151091,-1.697925


#### 범주형 데이터 인코딩

In [ ]:
categoric_col = train.select_dtypes(include=['object']).columns
categoric_col

Index(['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender'], dtype='object')

In [ ]:
train = pd.get_dummies(columns=categoric_col, data=train)
train

,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_grams,Warehouse_block_A,Warehouse_block_B,Warehouse_block_C,Warehouse_block_D,Warehouse_block_F,Mode_of_Shipment_Flight,Mode_of_Shipment_Road,Mode_of_Shipment_Ship,Product_importance_high,Product_importance_low,Product_importance_medium,Gender_F,Gender_M
3,-0.972147,0.005764,-0.786512,0.239733,1.212500,-1.718296,0,1,0,0,0,1,0,0,0,0,1,0,1
5,-0.972147,-1.412904,-1.077646,-0.420175,1.803398,-1.570146,0,0,0,0,1,1,0,0,0,0,1,1,0
6,-0.972147,0.715099,0.752342,-0.420175,-0.855642,-0.981252,0,0,0,1,0,1,0,0,0,1,0,1,0
8,-0.972147,0.715099,-1.327190,-0.420175,1.507949,-1.296070,1,0,0,0,0,1,0,0,0,1,0,1,0
10,-0.972147,0.715099,-0.516173,-1.080084,1.803398,-0.662113,0,0,1,0,0,1,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10994,-0.113596,-1.412904,0.793933,0.899642,-1.446539,-1.495454,1,0,0,0,0,0,0,1,0,0,1,1,0
10995,-0.113596,-1.412904,0.378027,0.899642,0.030705,-1.675086,0,1,0,0,0,0,0,1,0,0,1,1,0
10996,0.744955,0.715099,0.585980,0.899642,-0.560193,-1.731876,0,0,1,0,0,0,0,1,0,1,0,1,0
10997,0.744955,-0.703570,0.190869,1.559551,-1.151091,-1.697925,0,0,0,0,1,0,0,1,0,0,1,0,1


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train, target, stratify=target, random_state=13, test_size=0.2)

### 모델학습

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5600686498855835

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
dtc.score(X_test, y_test)

0.5594965675057209

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


0.584096109839817

In [ ]:
pred = rf.predict(X_test)
pred

array([0, 0, 0, ..., 1, 0, 0])

In [ ]:
result = pd.DataFrame({
    'is_reached_on_time': pred
})

result

,is_reached_on_time
0,0
1,0
2,0
3,0
4,1
...,...
1743,1
1744,0
1745,1
1746,0
